# 🚀 自研动态图学习框架 AdaptoFlux：在 Titanic 上实现全自动建模

欢迎使用 **AdaptoFlux** —— 一种基于函数图自动生长的可解释机器学习框架。

本 Notebook 演示如何：
- 加载预处理数据
- 训练动态函数图模型
- 保存图结构为 JSON
- 可视化图结构（交互式）
- 生成 Kaggle 提交文件

👉 所有代码开源，欢迎 Star & Fork！

In [ ]:
import pandas as pd
import numpy as np
import json
import traceback
from ATF.core.flux import AdaptoFlux
from ATF.CollapseManager.collapse_functions import CollapseMethod
from ATF.ModelTrainer.LayerGrowTrainer.layer_grow_trainer import LayerGrowTrainer

import logging

logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s] %(name)s: %(message)s'
)

## 🧩 自定义函数库（methods.py）

确保 `methods.py` 已定义基础函数（如 `add_values`, `multiply_values` 等），并用 `@method_profile` 装饰。

## 🗃️ 加载 Titanic 预处理数据

In [ ]:
def load_titanic_for_adaptoflux(train_processed_path, methods_path=None, collapse_method=CollapseMethod.SUM):
    df = pd.read_csv(train_processed_path)
    if 'Survived' not in df.columns:
        raise ValueError("train_processed.csv 必须包含 'Survived' 列")
    labels = df['Survived'].values
    values = df.drop(columns=['Survived']).values
    values = np.array(values, dtype=np.float64)
    return AdaptoFlux(
        values=values,
        labels=labels,
        methods_path=methods_path,
        collapse_method=collapse_method
    )

# 加载模型
model = load_titanic_for_adaptoflux(
    train_processed_path='examples/kaggle/titanic/output/train_processed.csv',
    methods_path='examples/kaggle/titanic/methods.py',
    collapse_method=CollapseMethod.SUM
)

print(f"✅ 加载数据完成：{model.values.shape[0]} 个样本，{model.values.shape[1]} 个特征")

## 🌱 添加自定义坍缩函数

In [ ]:
def collapse_sum_positive(values):
    total = np.sum(values)
    return 1 if total > 0 else 0

model.add_collapse_method(collapse_sum_positive)

## 🏗️ 模型训练（逐层生长）

In [ ]:
trainer = LayerGrowTrainer(
    adaptoflux_instance=model,
    max_attempts=10,
    decision_threshold=0.0,
    verbose=True
)

trainer.train(
    input_data=model.values,
    target=model.labels,
    max_layers=15,
    save_model=True,
    on_retry_exhausted="rollback",
    rollback_layers=2,
    max_total_attempts=2000
)

## 💾 保存图结构为 JSON

In [ ]:
model.save_graph("titanic_graph.json")
print("✅ 图结构已保存为 titanic_graph.json")

## 🎨 可视化图结构（交互式）

In [ ]:
!pip install pyvis -q

In [ ]:
from pyvis.network import Network
import json

def visualize_graph_interactive(json_path, output_html="adaptoflux_graph.html"):
    net = Network(directed=True, height="800px", width="100%", bgcolor="#f9f9f9", font_color="#222")

    with open(json_path, 'r') as f:
        graph_data = json.load(f)

    for node in graph_data['nodes']:
        node_id = node['id']
        label = node.get('name', f"Node_{node_id}")
        title = f"<b>ID:</b> {node_id}<br><b>Type:</b> {node['type']}"><br><b>Func:</b> {label}" if 'name' in node else str(node)

        if node['type'] == 'input':
            color = 'lightgreen'
            shape = 'dot'
        elif node['type'] == 'function':
            color = 'lightblue'
            shape = 'square'
        elif 'collapse' in label.lower() or node['type'] == 'output':
            color = 'salmon'
            shape = 'diamond'
        else:
            color = 'gray'
            shape = 'dot'

        net.add_node(node_id, label=label, color=color, shape=shape, title=title)

        if 'inputs' in node:
            for inp in node['inputs']:
                if isinstance(inp, dict) and 'source' in inp:
                    src_id = inp['source']
                    net.add_edge(src_id, node_id)
                elif isinstance(inp, int):
                    net.add_edge(inp, node_id)

    net.set_options('''
    var options = {
      "physics": {
        "enabled": true,
        "repulsion": { "nodeDistance": 120, "springLength": 100 }
      },
      "interaction": { "hover": true, "tooltip": true }
    }
    ''')

    net.show(output_html)
    return net

# 生成可视化
viz = visualize_graph_interactive("titanic_graph.json", "adaptoflux_titanic.html")
print("✅ 交互式图已生成：adaptoflux_titanic.html")
viz  # 在 notebook 中显示

## 📊 模型评估

In [ ]:
def _evaluate_accuracy(output: np.ndarray, target: np.ndarray) -> float:
    try:
        if len(output.shape) == 1 or output.shape[1] == 1:
            pred_classes = (output >= 0.5).astype(int).flatten()
        else:
            pred_classes = np.argmax(output, axis=1)
        true_labels = np.array(target).flatten()
        return np.mean(pred_classes == true_labels)
    except Exception as e:
        print(f"评估失败: {e}")
        return 0.0

pred = model.infer_with_graph(model.values)
acc = _evaluate_accuracy(pred, model.labels)
print(f"🎯 最终准确率: {acc:.4f}")

## 📤 生成 Kaggle 提交文件

In [ ]:
passenger_ids = range(892, 892 + len(pred))
submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': (pred >= 0.5).astype(int).flatten()
})
submission.to_csv('submission.csv', index=False)
print("✅ 提交文件已生成：submission.csv")
display(submission.head())

## 🙏 致谢

感谢 Kaggle 提供 Titanic 数据集。

如果你喜欢这个项目，请：
- ⭐ Star 仓库
- 🔄 提交 PR 添加新函数
- 💬 在评论区分享你的想法！

我们一起打造更透明、更可控的 AI 未来。